In [1]:
from bs4 import BeautifulSoup
import requests
import pymongo
import pandas as pd

In [39]:
conn = 'mongodb://localhost:27017'
client = pymongo.MongoClient(conn)

In [ ]:
db = client.vg_reviews_db

In [ ]:
db.drop_collection("rev_scrape")

In [40]:
collection = db.rev_scrape

In [2]:
sdf = pd.read_csv("vgsales.csv")
sdf.head()

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
0,1,Wii Sports,Wii,2006.0,Sports,Nintendo,41.49,29.02,3.77,8.46,82.74
1,2,Super Mario Bros.,NES,1985.0,Platform,Nintendo,29.08,3.58,6.81,0.77,40.24
2,3,Mario Kart Wii,Wii,2008.0,Racing,Nintendo,15.85,12.88,3.79,3.31,35.82
3,4,Wii Sports Resort,Wii,2009.0,Sports,Nintendo,15.75,11.01,3.28,2.96,33.00
4,5,Pokemon Red/Pokemon Blue,GB,1996.0,Role-Playing,Nintendo,11.27,8.89,10.22,1.00,31.37


In [3]:
sdf.loc[sdf["Platform"] == "SAT"]

,Rank,Name,Platform,Year,Genre,Publisher,NA_Sales,EU_Sales,JP_Sales,Other_Sales,Global_Sales
888,890,Virtua Fighter 2,SAT,1995.0,Fighting,Sega,0.34,0.26,1.30,0.03,1.93
1753,1755,Sega Rally Championship,SAT,1995.0,Racing,Sega,0.21,0.16,0.77,0.02,1.16
1923,1925,Virtua Fighter,SAT,1994.0,Fighting,Sega,0.17,0.12,0.77,0.02,1.07
3237,3239,Virtua Cop,SAT,1995.0,Shooter,Sega,0.00,0.00,0.62,0.00,0.62
3274,3276,Fighters MEGAMiX,SAT,1996.0,Fighting,Sega,0.00,0.00,0.62,0.00,0.62
...,...,...,...,...,...,...,...,...,...,...,...
14927,14930,America Oudan Ultra-Quiz,SAT,1995.0,Misc,Victor Interactive,0.00,0.00,0.02,0.00,0.02
15185,15188,Shippuu Mahou Daisakusen: Kingdom-Grandprix,SAT,1996.0,Shooter,Gaga,0.00,0.00,0.02,0.00,0.02
15256,15259,Pebble Beach Golf Links,SAT,1994.0,Sports,Sega,0.00,0.00,0.02,0.00,0.02
15289,15292,NightTruth: Explanation of the Paranormal - Th...,SAT,1996.0,Misc,Sonnet,0.00,0.00,0.02,0.00,0.02


In [4]:
sdf["Platform"].value_counts()

DS      2163
PS2     2161
PS3     1329
Wii     1325
X360    1265
PSP     1213
PS      1196
PC       960
XB       824
GBA      822
GC       556
3DS      509
PSV      413
PS4      336
N64      319
SNES     239
XOne     213
SAT      173
WiiU     143
2600     133
NES       98
GB        98
DC        52
GEN       27
NG        12
SCD        6
WS         6
3DO        3
TG16       2
PCFX       1
GG         1
Name: Platform, dtype: int64

Not all of these platforms are available on metacritic.com
This is a list of the platforms that will be dropped from this data in order to scrape from metacritic
NES, SNES, SAT, 2600, GB, GEN, NG, SCD, WS, 3DO, TG16, GG, PCFX

In [22]:
def urlmaker(platform, name):
    platform_dict = {
        "ds": "ds",
        "ps2":"playstation-2",
        "ps3": "playstation-3",
        "wii": "wii",
        "x360": "xbox-360",
        "psp": "psp",
        "ps": "playstation",
        "pc": "pc",
        "xb": "xbox",
        "gba": "game-boy-advance",
        "gc": "gamecube",
        "3ds": "3ds",
        "psv": "playstation-vita",
        "ps4": "playstation-4",
        "n64": "nintendo-64",
        "xone": "xbox-one",
        "wiiu": "wii-u",
        "dc": "dreamcast",
    }
    
    platform_fixed = platform_dict[platform.lower()]
    
    name_fixed = name.replace(" ", "-")
    name_fixed = name_fixed.lower()
    
    return f"http://metacritic.com/game/{platform_fixed}/{name_fixed}"

In [23]:
url_list = []
for row in sdf[["Platform", "Name"]].iterrows():
    try:
        url = urlmaker(row[1]["Platform"], row[1]["Name"])
        url_list.append(url)
    except: 
        pass
    
url_list

['http://metacritic.com/game/wii/wii-sports',
 'http://metacritic.com/game/wii/mario-kart-wii',
 'http://metacritic.com/game/wii/wii-sports-resort',
 'http://metacritic.com/game/ds/new-super-mario-bros.',
 'http://metacritic.com/game/wii/wii-play',
 'http://metacritic.com/game/wii/new-super-mario-bros.-wii',
 'http://metacritic.com/game/ds/nintendogs',
 'http://metacritic.com/game/ds/mario-kart-ds',
 'http://metacritic.com/game/wii/wii-fit',
 'http://metacritic.com/game/wii/wii-fit-plus',
 'http://metacritic.com/game/xbox-360/kinect-adventures!',
 'http://metacritic.com/game/playstation-3/grand-theft-auto-v',
 'http://metacritic.com/game/playstation-2/grand-theft-auto:-san-andreas',
 'http://metacritic.com/game/ds/brain-age:-train-your-brain-in-minutes-a-day',
 'http://metacritic.com/game/ds/pokemon-diamond/pokemon-pearl',
 'http://metacritic.com/game/xbox-360/grand-theft-auto-v',
 'http://metacritic.com/game/playstation-2/grand-theft-auto:-vice-city',
 'http://metacritic.com/game/game

In [32]:
response = requests.request("GET", url_list[0], headers={"User-Agent": "Mozilla/5.0"})
soup = BeautifulSoup(response.text, "lxml")
reviews = soup.find_all("li", class_="critic_review")
# reviews[0].find("a", class_="external").text

In [46]:
reviews[0].find("div", class_="review_body").text.strip()

'For my money, I think I actually like this one better than "Winning Eleven 7." FIFA is packed with more features than you could possibly explore unless you play nothing but this for the next six months. [Nov 2004, p.138]'

In [45]:
rdf_list = []
for game_url in url_list:
    try:
        response = requests.request("GET", game_url, headers={"User-Agent": "Mozilla/5.0"})
        soup = BeautifulSoup(response.text, "lxml")
        
        reviews = soup.find_all("li", class_="critic_review")
        
        for review in reviews:
            
            title = soup.find("div", class_="product_title").find("h1").text
            source = review.find("div", class_="source").text
            score = review.find("div", class_="review_grade").text.strip()
            body =  review.find("div", class_="review_body").text.strip()
            
            if (title and source and score and body):
                print("---------------------")
                print(title)
                print(source)
                print(score)
                print(body)
                
            review_data = {
                "Game Title": title,
                "Review Source": source,
                "Review Score": score,
                "Review Body": body
            }
            
            rdf_list.append(review_data)
    except Exception as e:
        print(e)
            

---------------------
Wii Sports
Official Nintendo Magazine UK

90


                                The ideal way to get into Wii gaming. It's funny, simple and approachable, and gamers and non-gamers alike should get something out of it. [Christmas 2006, p.62]
                            
---------------------
Wii Sports
GameZone

85


                                Sure, it lacks goals and can be beaten very quickly. But the ultra-responsive technology and high multiplayer replay value are far greater than any other party or sports game collection.
                            
---------------------
Wii Sports
GameDaily

80


                                Wii Sports certainly won't be hailed as the most realistic sports sim on the market, but it's the best thing - next to a bottle of tequila - to take to a friend's holiday house party.
                            
---------------------
Wii Sports
GamingExcellence

77


                                Despite these two solo modes, 

---------------------
Mario Kart DS
GameSpy

100


                                The online shortcomings are annoying, yes, but very minor, and they are more than made up for by the amazing single-player content and local WiFi gameplay.
                            
---------------------
Mario Kart DS
Gamenikki

100


                                Mario Kart DS takes the best of all of the Mario Kart games and puts them together in a game people will still be playing when the next generation of Gameboy comes out.
                            
---------------------
Mario Kart DS
Play Magazine

95


                                Probably the best Mario Kart game since the original. [Dec 2005, p.75]
                            
---------------------
Mario Kart DS
Adrenaline Vault

90


                                It is refreshing to see the DS stand up and do some things that can't be done on the GBA, allowing it to stand out as its own system rather than being used to play GBA ti

---------------------
Grand Theft Auto V
LEVEL (Czech Republic)

100


                                GTA V fulfills the idea of a gaming sandbox, where you can entertain yourself in many different ways - and also experience a superbly written story of three gangsters, if you want. [Issue#234]
                            
---------------------
Grand Theft Auto V
Eurogamer Spain

100


                                GTA5 does much more than repeat the San Andreas model, introducing many new features in the franchise. From those, the one that adds a whole new dimension to the game is the fact that for the first time we have three main characters instead of one. GTA is the king of controversy... long live the king.
                            
---------------------
Grand Theft Auto V
Eurogamer Italy

100


                                Rockstar shows one more time how to build a masterpiece. GTA V is not perfect but has everything you'll ever dream in a sandbox game. San Andreas is go

---------------------
Halo 3
GamePro

100


                                It's solid gameplay, immense replayability, online functionality and incredible production values will ensure its place in video game history, and it is, without a doubt, a satisfying and fulfilling close to the beloved trilogy[?].
                            
---------------------
Halo 3
Edge Magazine

100


                                In substance it's nothing new, merely a magnificent, beautiful monster of an FPS sequel. In concept and execution, though, Halo 3 is the future. [Nov 2007, p.80]
                            
---------------------
Halo 3
NZGamer

99


                                Halo 3 is essential. An Xbox 360 without a copy of the game is like a human without a heart.
                            
---------------------
Halo 3
Extreme Gamer

95


                                Halo 3 gives a deeper, smoother and grander ride then ever before.
                            
----------------

---------------------
Grand Theft Auto IV
Official Xbox Magazine UK

100


                                It's got a world you believe in, a cast you care about and a script stuffed with brilliant moments... Utterly stunning in every resepct. [May 2008, p.79]
                            
---------------------
Grand Theft Auto IV
PGNx Media

100


                                Grand Theft Auto IV is undoubtedly one of the strongest games of this generation. The amount of content is simply mind-blowing.
                            
---------------------
Grand Theft Auto IV
ZTGD

100


                                I could go on for another couple thousand words about the amazing experience that is GTA IV, but what's most important for you to know is that this game is an absolute masterpiece.
                            
---------------------
Grand Theft Auto IV
VideoGamer

100


                                We all knew GTA 4 would be a great game and sell millions of copies, but 

...

---------------------
Minecraft: PlayStation 4 Edition
IGN

97


                                That sense of creative progression, coupled with the inherent danger of exploring underground caverns full of monsters, makes Minecraft exciting, rewarding, tense, and one of gaming’s most expressive creative outlets.
                            
---------------------
Minecraft: PlayStation 4 Edition
Impulsegamer

95


                                The Minecraft PlayStation 4 Edition is the penultimate version of this game on any console.
                            
---------------------
Minecraft: PlayStation 4 Edition
CD-Action

90


                                It’s the best console version of one of the most important games of the decade. Some limitations imposed by the platform might annoy hardcore Minecraft fans, but they are not the target group here. [13/2014, p.42]
                            
---------------------
Minecraft: PlayStation 4 Edition
Games Master UK

89


      

---------------------
EA Sports Active
GameFocus

91


                                EA hasn’t marketed their new program for those looking to lose weight or get in shape; rather it’s been marketed to help keep people active, hence the name, EA Sports Active. Despite this, some will still add it as part of their weight loss program, some will still use it to help them get in shape (guilty as charged), other’s will see it as a good reason to stay fit. Whatever you’re reason, EA may have unlocked the way to get gamers’ off the couch and maybe, save us some money on gym fee’s during the current economy.
                            
---------------------
EA Sports Active
GamePro

90


                                But EA Sports Active has a great core--it will motivate you to work out and have you doing it the right way.
                            
---------------------
EA Sports Active
Gameplanet

85


                                We were well impressed with this one; it offers gr

---------------------
Sports Champions
Gaming Age

91


                                I honestly wasn't expecting much, but the precise, enjoyably implemented motion controls and event variety totally sold me.
                            
---------------------
Sports Champions
Impulsegamer

83


                                At the end of the day,  Sports Champions is a must have release game for the PlayStation Move that captures the potential of this new control system perfectly for the PlayStation 3. 
                            
---------------------
Sports Champions
GamesRadar+

80


                                Fantastic showcase for Move.
                            
---------------------
Sports Champions
GameShark

75


                                Sports Champions is a good opening effort for the PlayStation Move. It's not going to capture the attention of the world the way Wii Sports did, but it has put to rest any speculation that Sony's technology isn't up to snuf

---------------------
FIFA Soccer 2005
Official U.S. Playstation Magazine

100


                                For my money, I think I actually like this one better than "Winning Eleven 7." FIFA is packed with more features than you could possibly explore unless you play nothing but this for the next six months. [Nov 2004, p.138]
                            
---------------------
FIFA Soccer 2005
PGNx Media

90


                                A very solid product overall. The game’s presentation is superb, the gameplay is good and fast-paced, and the game offers much replayability.
                            
---------------------
FIFA Soccer 2005
GameSpot

88


                                There's no doubt that this is EA Sports' best soccer game to date on the pitch, but it's really unfortunate that the career mode isn't more user-friendly.
                            
---------------------
FIFA Soccer 2005
PSM Magazine

80


                                With depth and sty

KeyboardInterrupt: 

In [26]:
collection.insert_many(rdf_list)

'playstation-2'

In [47]:
rdf = pd.DataFrame(rdf_list)
rdf.head()

,Game Title,Review Source,Review Score,Review Body,_id
0,Wii Sports,Official Nintendo Magazine UK,\n90\n,\n The ideal wa...,5f21f9505840fa70db006823
1,Wii Sports,GameZone,\n85\n,"\n Sure, it lac...",5f21f9505840fa70db006824
2,Wii Sports,GameDaily,\n80\n,\n Wii Sports c...,5f21f9505840fa70db006825
3,Wii Sports,GamingExcellence,\n77\n,\n Despite thes...,5f21f9505840fa70db006826
4,Wii Sports,My Gamer,\n74\n,\n This is one ...,5f21f9505840fa70db006827
